### Fitting DCC GARCH model using R - switch kernel

In [1]:
#install.packages("rmgarch")
library(rmgarch)
library(parallel)
library(quantmod)
library(tidyverse)
library(xts)
library(ggplot2)
library(reshape2)

Loading required package: rugarch

Loading required package: parallel


Attaching package: ‘rugarch’


The following object is masked from ‘package:stats’:

    sigma


Loading required package: xts

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘xts’


The following objects are masked from ‘package:rmgarch’:

    first, last


Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Found more than one class "atomicVector" in cache; using the first, from namespace 'Matrix'

Also defined by ‘Rmpfr’

Found more than one class "atomicVector" in cache; using the first, from namespace 'Matrix'

Also defined by ‘Rmpfr’

Found more than one class "atomicVector" in cache; using the first, from namespace 'Matrix'

Also defined by ‘Rmpfr’

Found more than one class "atomicVector" in cache; using the first,

In [2]:
etfs <- read.csv("../data/etfs.csv", sep=";")

In [5]:
#etfs <- etfs %>% select(-Date)
etfs <- etfs %>% mutate(Date=as.Date(Date, format = "%Y-%m-%d"))

### Converting dataset to xts

In [6]:
xts_etfs <- xts(etfs, order.by = etfs[,1])

In [7]:
xspec = ugarchspec(mean.model = list(armaOrder = c(0, 0)), variance.model = list(garchOrder = c(1,1), model = 'sGARCH'), distribution.model = 'norm')
uspec = multispec(replicate(10, xspec))
spec1 = dccspec(uspec = uspec, dccOrder = c(1, 1), distribution = 'mvnorm')
cl = makePSOCKcluster(4)
multf = multifit(uspec, xts_etfs[,2:11], cluster = cl)
fit1 = dccfit(spec1, data = xts_etfs[,2:11], fit.control = list(eval.se = TRUE), fit = multf, cluster = cl)
print(fit1)           
stopCluster(cl)


*---------------------------------*
*          DCC GARCH Fit          *
*---------------------------------*

Distribution         :  mvnorm
Model                :  DCC(1,1)
No. Parameters       :  87
[VAR GARCH DCC UncQ] : [0+40+2+45]
No. Series           :  10
No. Obs.             :  2523
Log-Likelihood       :  -29794.13
Av.Log-Likelihood    :  -11.81 

Optimal Parameters
-----------------------------------
              Estimate  Std. Error   t value Pr(>|t|)
[EEM].mu      0.034180    0.021635   1.57982 0.114149
[EEM].omega   0.058735    0.017199   3.41510 0.000638
[EEM].alpha1  0.121898    0.021940   5.55594 0.000000
[EEM].beta1   0.844901    0.026915  31.39088 0.000000
[EWZ].mu      0.017190    0.035890   0.47896 0.631971
[EWZ].omega   0.105846    0.047113   2.24664 0.024663
[EWZ].alpha1  0.099854    0.026088   3.82760 0.000129
[EWZ].beta1   0.879802    0.030233  29.10076 0.000000
[FXI].mu      0.041327    0.026493   1.55988 0.118787
[FXI].omega   0.095212    0.049052   1.94107 0

In [10]:
rcov(fit1, output="matrix")

                  EEM    EWZ:EEM    FXI:EEM      GDX:EEM     HYG:EEM
2011-09-01  1.8625995  2.2583440  1.8323355  0.934619660  0.44890390
2011-09-02  1.6723566  2.0308011  1.7224466  0.850518549  0.45337018
2011-09-06  2.1120367  2.4420475  2.1077610  0.837320798  0.49900328
2011-09-07  1.9824521  2.3024921  1.9288638  0.774005768  0.61842462
2011-09-08  2.7816318  2.7835811  2.3094291  0.868283532  0.69336102
2011-09-09  3.0281174  2.8018729  2.5572919  0.821117181  0.65063615
2011-09-12  4.0387992  3.7655370  3.1804170  0.971510215  0.84113618
2011-09-13  3.4778018  3.3818428  2.8019214  0.922639310  0.81406603
2011-09-14  2.9987756  2.9763227  2.4673717  0.834529029  0.75687046
2011-09-15  2.6040610  2.6442203  2.1834786  0.777020604  0.68603698
2011-09-16  2.5047646  2.4864008  2.0867801  0.725243157  0.66896749
2011-09-19  2.1773624  2.2037747  1.8708255  0.677455284  0.56454657
2011-09-20  2.9917728  2.6887761  2.6237755  0.801891865  0.59313325
2011-09-21  2.6590476  2.4547138  

In [35]:
#coef(fit1) %>% write.csv('coef.csv')
#install.packages("gmvarkit")
library(gmvarkit)

In [29]:
etf_forecast <- dccforecast(fit1, n.ahead=1)

In [39]:
rcov(etf_forecast, output="matrix")

,EEM,EWZ:EEM,FXI:EEM,GDX:EEM,HYG:EEM,SLV:EEM,SPY:EEM,TLT:EEM,XLF:EEM,XLU:EEM,⋯,SPY,TLT:SPY,XLF:SPY,XLU:SPY,TLT,XLF:TLT,XLU:TLT,XLF,XLU:XLF,XLU
T+1,0.008088057,-6.194942e-05,0.01780162,0.01094251,0.02511785,0.002839959,0.08531867,0.03429511,0.08243349,0.08448728,⋯,0.9132665,0.3688015,0.8818356,0.9057751,0.1500154,0.3559758,0.3666705,0.8522604,0.8746033,0.8998687


In [50]:
rcov(etf_forecast, output="matrix")

,EEM,EWZ:EEM,FXI:EEM,GDX:EEM,HYG:EEM,SLV:EEM,SPY:EEM,TLT:EEM,XLF:EEM,XLU:EEM,⋯,SPY,TLT:SPY,XLF:SPY,XLU:SPY,TLT,XLF:TLT,XLU:TLT,XLF,XLU:XLF,XLU
T+1,0.008088057,-6.194942e-05,0.01780162,0.01094251,0.02511785,0.002839959,0.08531867,0.03429511,0.08243349,0.08448728,⋯,0.9132665,0.3688015,0.8818356,0.9057751,0.1500154,0.3559758,0.3666705,0.8522604,0.8746033,0.8998687


In [ ]:
etf_forecast

### Using DCC-roll to forecast

In [104]:
cl = makePSOCKcluster(8)
roll <- dccroll(spec = spec1, data = xts_etfs[,2:11], forecast.length = 1, n.ahead=1, refit.every = 1,
               refit.window = "recursive")

#,             fit.control = list(eval.se = TRUE, stationarity = TRUE, scale = FALSE))
stopCluster(cl)

Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”


In [99]:
data <- as_tibble(rcov(roll, output="matrix")) %>%
        mutate(Date = seq.int(1, nrow(rcov(roll, output="matrix")), 1), .before=EEM) 

print(data)

# A tibble: 25 × 56
    Date   EEM `EWZ:EEM` `FXI:EEM` `GDX:EEM` `HYG:EEM` `SLV:EEM` `SPY:EEM`
   <dbl> <dbl>     <dbl>     <dbl>     <dbl>     <dbl>     <dbl>     <dbl>
 1     1 1.62       1.91      2.53     0.670     0.217     0.619     0.524
 2     2 1.46       1.73      2.30     0.615     0.195     0.567     0.492
 3     3 1.29       1.57      2.06     0.581     0.179     0.529     0.457
 4     4 1.25       1.45      1.94     0.642     0.167     0.624     0.412
 5     5 1.12       1.31      1.81     0.610     0.156     0.627     0.366
 6     6 1.01       1.19      1.64     0.566     0.146     0.574     0.327
 7     7 0.917      1.08      1.49     0.548     0.135     0.538     0.299
 8     8 0.935      1.12      1.48     0.565     0.129     0.555     0.289
 9     9 0.859      1.02      1.35     0.534     0.122     0.529     0.267
10    10 0.893      1.08      1.37     0.532     0.116     0.510     0.262
# … with 15 more rows, and 48 more variables: TLT:EEM <dbl>, XLF:EEM <dbl>,
#   

In [ ]:
data %>% ggplot(aes(x = Date)) +
geom_line(color = Series) +
# Add a second axis and specify its features
labs(
  title="Macroeconomic indicators",
  x = "Year",
  y = "Count")


#rcov(roll, output="matrix")